# Задача на 6-ом шаге урока

**🪢 Пишем categorical_transformer. 🤖**

Задача:

*   При помощи sklearn.Pipelines напишите свой categorical_transformer, который принимает на вход pd.DataFrame

*   Состоящий только из категориальных признаков

*   Заполняет пропуски строкой 'Unknown', а потом делает one hot encoding.






Примечание: Не нужно писать это руками, воспользуйтесь трансформерами из sklearn.impute и sklearn.preprocessing.

*Для отладки решения можно использовать следующий датасет:*

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

data_root = "https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/"
data = pd.read_csv(data_root + 'quickstart_train.csv')

cols2drop = ['car_id', 'target_reg', 'target_class']
categorical_features = ['model', 'car_type', 'fuel_type']
numerical_features = [c for c in data.columns
                      if c not in categorical_features and c not in cols2drop]

Решение:

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline


# Создаем pipeline для обработки категориальных признаков
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])
data_pred = categorical_transformer.fit_transform(data[categorical_features])

# Задача на 7-ом шаге урока

🧵 Пишем numerical_transformer 🤖

Задача:

* Напишите свой numerical_transformer, который принимает на вход pd.DataFrame, состоящий только из численных признаков.

* Он должен заполнить пропуски медианой, потом нормализовать данные с помощью MinMaxScaler

* отобрать топ-5 признаков с наибольшим
значением функции [ANOVA F-value](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html).



Примечание: Не нужно писать это руками, воспользуйтесь трансформерами из sklearn.

*Для отладки решения можно использовать следующий датасет:*

In [3]:
import pandas as pd

data_root = "https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/"
data = pd.read_csv(data_root + 'quickstart_train.csv')

Решение:

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif

# Создаем pipeline
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),  # Заполнение пропусков медианой
    ("scaler", MinMaxScaler()),  # Нормализация данных
    ("feature_selector", SelectKBest(score_func=f_classif, k=5))  # Выбор топ-5 признаков
])

data_pred = numerical_transformer.fit_transform(data[numerical_features],
                                                data['target_reg'])

print(data[numerical_features].shape)
print(data_pred.shape)

(2337, 11)
(2337, 5)


# Задача на 8-ом шаге урока

**🧳 Пишем data_transformer**

Задача:

*  Объедините предыдущие два шага вместе чтобы выполнялось следующее

*  на numerical_features ваш трансформер вызывал numerical_transformer

*  на categorical_features использовал categorical_transformer.


Примечание:  categorical_transformer и numerical_transformer считайте уже объявленными.



*Решение тестируется на тех же данных, что и предыдуще задачи - используются  трансформеры из предыдущих задач на 6-ом и 7-ом уроке*

Решение:

In [5]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif

import warnings

warnings.filterwarnings("ignore")


cols2drop = ['car_id', 'target_reg', 'target_class']
categorical_features = ['model', 'car_type', 'fuel_type']
numerical_features = [c for c in data.columns
                      if c not in categorical_features and c not in cols2drop]

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),  # Заполнение пропусков медианой
    ("scaler", MinMaxScaler()),  # Нормализация данных
    ("feature_selector", SelectKBest(score_func=f_classif, k=5))  # Выбор топ-5 признаков
])

# соединим два предыдущих трансформера в один
data_transformer = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    ("categorical", categorical_transformer, categorical_features)])

data_pred = data_transformer.fit_transform(data, data['target_reg'])

print(data[numerical_features].shape)
print(data_pred.shape)

(2337, 11)
(2337, 37)


# Задача на 9-ом шаге урока

**🌂 Пишем свой классификатор**

Задача:

* Написать свой ансамбль по приведенной ниже структуре
<center> <img src = https://ucarecdn.com/94fb93eb-b945-4cdf-900f-6741c1b0aaa4/> </center>

*   В качестве final_estimator используй LogisticRegression()

Примечание: ColumnTransformer уже объявлен и лежит в переменной data_transofmer. Список моделей вы можете найти в словаре в коде (их объявлять и импортировать не нужно).

*Для тестирования решения можно использовать data_transformer из прошлой задачи*

Решение:

In [6]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

import lightgbm as lgbm
import xgboost as xgb
import catboost as cb

import warnings

warnings.filterwarnings("ignore")


cols2drop = ['car_id', 'target_reg', 'target_class']
categorical_features = ['model', 'car_type', 'fuel_type']
numerical_features = [c for c in data.columns
                      if c not in categorical_features and c not in cols2drop]

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),  # Заполнение пропусков медианой
    ("scaler", MinMaxScaler()),  # Нормализация данных
#     ("feature_selector", SelectKBest(score_func=f_classif, k=5))  # Выбор топ-5 признаков
])

# соединим два предыдущих трансформера в один
data_transformer = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    ("categorical", categorical_transformer, categorical_features)])

params_cat = {'verbose': False,
              'use_best_model': True,
              'cat_features': categorical_features,
              'loss_function': 'MultiClass',
              'random_state': 42,
              }
cat_model = cb.CatBoostClassifier(**params_cat)

params_lgbm = {'cat_features': categorical_features,
               'random_state': 42,
               'verbosity': -1,
               }
lgbm_model = lgbm.LGBMClassifier(**params_lgbm)

params_xgb = {'enable_categorical': True
              }
xgb_model = xgb.XGBClassifier(**params_xgb)

random_forest_model = RandomForestClassifier(warm_start=True,
                                             n_jobs=-1,
                                             random_state=75,
                                             verbose=False)

models = {'random_forest': random_forest_model,
          'xgboost': xgb_model,
          'lightgbm': lgbm_model,
          'catboost': cat_model}

estimators = [("random_forest", make_pipeline(data_transformer, random_forest_model)),
              ("xgboost", make_pipeline(data_transformer, xgb_model)),
              ("lightgbm", lgbm_model),
              ("catboost", cat_model),
              ]

# в качестве мета-модели будем использовать LogisticRegression
stacking_classifier = StackingClassifier(estimators=estimators,
                                         final_estimator=LogisticRegression(verbose=False),
                                         n_jobs=-1,
                                         verbose=False,
                                         )

stacking_classifier

StackingClassifier(estimators=[('random_forest',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('numerical',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer(strategy='median')),
                                                                                                  ('scaler',
                                                                                                   MinMaxScaler())]),
                                                                                  ['car_rating',
                                                                                   'year_to_start',
                                                                                   'riders',
                                                                                   'year_to_work',
                                                                                   'mean_rating',
                                                                                   'distance_sum',
                                                                                   'rating_min',
                                                                                   'speed_max',
                                                                                   'user_ride_quality_medi...
                                                               multi_strategy=None,
                                                               n_estimators=None,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=None, ...))])),
                               ('lightgbm',
                                LGBMClassifier(cat_features=['model',
                                                             'car_type',
                                                             'fuel_type'],
                                               random_state=42, verbosity=-1)),
                               ('catboost',
                                <catboost.core.CatBoostClassifier object at 0x000002799FFEC550>)],
                   final_estimator=LogisticRegression(verbose=False), n_jobs=-1,
                   verbose=False)

# Задача на 10-ом шаге урока

**🧶 Hard*. Пишем свой text_transformer 🤖**

Задача:



* Необходимо реализовать кастомный text_transformer, который можно будет встраивать в sklearn.Pipelines(). Подробно про то, как реализовать свой трансформер, вы можете прочитать [здесь](https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156).


*   трансформер должен работать с текстовыми столбцами и делать следующие преобразования:
1. Удалять из текста стоп-слова, которые задаются списком в параметрах трансформера (сравнение нужно проводить до нормализации).
2. Нормализовать слова при помощи get_normal_string(). То есть каждое слово нужно заменить на get_normal_string(word) для всех слов в тексте.
3. Cчитайте, что слова разделены пробелами.

*Ваш код будет тестироваться вот таким небольшим пайплайном (можете его использовать, чтобы отлаживаться локально):*

Решение:

In [7]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


# Заглушка для нормализации слов
def get_normal_string(word):
    # Здесь должна быть функция нормализации
    return word.lower()


class TextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, stop_words):
        """
        Инициализация трансформера.
        :param stop_words: Список стоп-слов.
        """
        self.stop_words = set(stop_words)  # Преобразуем в множество для быстрого поиска

    def fit(self, X, y=None):
        """
        Обучение трансформера (здесь ничего не нужно).
        """
        return self

    def filter_text(self, text):
        """
        Обработка одного текста: удаление стоп-слов и нормализация.
        :param text: Входной текст (строка).
        :return: Преобразованный текст (строка).
        """
        words = text.split()  # Разделяем текст на слова
        filtered_words = [get_normal_string(word) for word in words
                          if word not in self.stop_words]
        return " ".join(filtered_words)  # Объединяем обратно в строку

    def transform(self, X):
        """
        Преобразование данных.
        :param X: Входные данные (pd.DataFrame или np.array с текстовыми колонками).
        :return: Преобразованный массив той же размерности, что и входной.
        """
        if isinstance(X, pd.DataFrame):
            X = X.values  # Преобразуем в numpy массив

        transformed_data = np.array([self.filter_text(text) for text in X.ravel()])
        return transformed_data.reshape(X.shape)

In [8]:
# Данные для теста
data = pd.DataFrame({'text': ['текст пример один', 'ещё пример текста', 'пример текста']})
stop_words_list = ['пример', 'ещё']  # Стоп-слова

# Пайплайн с TextTransformer
text_transformers = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='constant', fill_value='')),  # Заполнение пропусков
    ("scaler", TextTransformer(stop_words_list))  # Наш кастомный трансформер
])

# Преобразование данных
transformed_data = text_transformers.fit_transform(data[['text']])
print(transformed_data)

[['текст один']
 ['текста']
 ['текста']]
